In [49]:
from langchain.document_loaders import UnstructuredPDFLoader, OnlinePDFLoader, PyPDFLoader

from langchain.text_splitter import RecursiveCharacterTextSplitter
import os
from dotenv import load_dotenv

load_dotenv()

False

In [87]:
os.environ["OPENAI_API_KEY"] = "api_key"

In [88]:
from langchain.document_loaders import DirectoryLoader
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.vectorstores import Chroma

In [89]:
loader = DirectoryLoader('f', glob="**/*.txt", show_progress=True)

docs = loader.load()

100%|█████████████████████████████████████████████| 1/1 [00:00<00:00, 32.56it/s]


In [90]:
print (f"You have {len(docs)} essays loaded")

You have 1 essays loaded


In [91]:
# Split
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1500, chunk_overlap=0)
splits = text_splitter.split_documents(docs)

print (f"Your {len(docs)} documents have been split into {len(splits)} chunks")

Your 1 documents have been split into 3 chunks


In [92]:
if 'vectordb' in globals(): 
    vectordb.delete_collection()

embedding = OpenAIEmbeddings()
vectordb = Chroma.from_documents(documents=splits, embedding=embedding)

In [93]:
from langchain.chat_models import ChatOpenAI
from langchain.retrievers.multi_query import MultiQueryRetriever
from langchain.prompts import PromptTemplate
import logging

In [94]:
logging.basicConfig()
logging.getLogger("langchain.retrievers.multi_query").setLevel(logging.INFO)

In [95]:
question = "What are the alternate ways to resolve this case?"
llm = ChatOpenAI(temperature=0)

retriever_from_llm = MultiQueryRetriever.from_llm(
    retriever=vectordb.as_retriever(), llm=llm
)

In [96]:
unique_docs = retriever_from_llm.get_relevant_documents(query=question)

INFO:langchain.retrievers.multi_query:Generated queries: ['1. What other methods can be used to solve this case?', '2. Are there any alternative approaches to resolving this case?', '3. Can you suggest different strategies for resolving this case?']


In [97]:
len(unique_docs)

3

In [98]:
prompt_template = """Use the following pieces of context to answer the question at the end.
If you don't know the answer, just say that you don't know, don't try to make up an answer.

{context}

Question: {question}
Answer:"""
PROMPT = PromptTemplate(
    template=prompt_template, input_variables=["context", "question"]
)

In [99]:
llm.predict(text=PROMPT.format_prompt(
    context=unique_docs,
    question=question
).text)

'The alternate ways to resolve this case could include litigation, arbitration, or negotiation outside of court.'